In [ ]:
import os
import pandas as pd
import csv
import numpy as np
import string
import torch

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# configurations
embedding_dim = 100
hidden_dim = 20
learning_rate = 0.01
batch_size = 1024
window_size = 3

In [ ]:
# fancy stuff
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [ ]:
wordToIndexDictionary = ''
with open('/content/gdrive/MyDrive/PyTorch/data/pa3_parta/wordToIndexDictionary.txt', 'r') as f:
    for i in f.readlines():
        wordToIndexDictionary = i

wordToIndexDictionary = eval(wordToIndexDictionary)

unique_words = []

with open('/content/gdrive/MyDrive/PyTorch/data/pa3_parta/unique_words.txt', 'r') as filehandle:
    for line in filehandle:
        unique_words.append(line[:-1])

vocab_size = len(unique_words)

trainset = pd.read_csv("/content/gdrive/MyDrive/PyTorch/data/pa3_parta/LSTM_Train.csv")
testset = pd.read_csv("/content/gdrive/MyDrive/PyTorch/data/pa3_parta/LSTM_Train.csv")

trainloader = torch.utils.data.DataLoader(trainset.values, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testset.values, batch_size=batch_size, shuffle=True)

In [ ]:
%cd '/content/gdrive/MyDrive/PyTorch'

/content/gdrive/MyDrive/PyTorch


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

In [ ]:
class LSTM(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, window_size):
        super(LSTM, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.linear = nn.Linear((window_size*4*hidden_dim), vocab_size)
        self.activation_function = nn.LogSoftmax(dim = -1)
        self.window_size = window_size
        self.hidden_dim = hidden_dim

    def forward(self, inputs):
        out = self.embeddings(inputs)
        out, _ = self.lstm(out)
        out = self.linear(out.reshape([out.shape[0],(self.window_size*4*self.hidden_dim)]))
        return self.activation_function(out)

    def get_word_emdedding(self, word):
        word = torch.tensor([wordToIndexDictionary[word]])
        return self.embeddings(word)

In [ ]:
lstm = LSTM(vocab_size, embedding_dim, hidden_dim, window_size)
lstm.to(device)
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

In [ ]:
def train(model, trainloader, val_iterator, optimizer, loss_function, epochs, results):
  
  for j in range(epochs):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    print(color.BOLD + "Training for epoch = " + color.END, j)
    for trainload in enumerate(tqdm(trainloader)):

        y_true = trainload[1][:,-1]

        x = trainload[1][:,:-1]
        y_pred = lstm.forward(x.to(device))

        loss = loss_function(y_pred, y_true.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    print(f"[{j}/{epochs}] : epoch_loss: {epoch_loss / len(trainloader):.4f}")
    val_loss = evaluate(model, val_iterator, loss_function)

    train_loss = epoch_loss / len(trainloader)
    results = results.append({"Epoch": (j+1), \
                            "Train Loss": train_loss, \
                            "Val Loss": val_loss},ignore_index=True)
    
    torch.save(model.state_dict(), f"./data/Sentiment_models/lstm/lstm_model_{j+1}.pth")
    results.to_csv(f"./data/Sentiment_models/lstm/lstm_Results_{embedding_dim}.csv")


  return train_loss

In [ ]:
def evaluate(model, testloader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    print("Validation")
    with torch.no_grad():
        for testload in enumerate(tqdm(testloader)):

            y_true = testload[1][:,-1]

            x = testload[1][:,:-1]
            y_pred = lstm.forward(x.to(device))

            loss = loss_function(y_pred, y_true.to(device))
            
            epoch_loss += loss.item()

    print(f"val_loss: {epoch_loss / len(testloader):.4f}")
    print("--------------------------------------------------------------------")
    print();print()
    return epoch_loss / len(testloader)

In [ ]:
epochs = 2
results = pd.DataFrame(columns=["Epoch","Train Loss","Val Loss"])
train(lstm, trainloader, testloader, optimizer, loss_function, epochs, results)

Training for epoch =  0



[0/2] : epoch_loss: 11.2397
Validation



val_loss: 10.6846
--------------------------------------------------------------------


Training for epoch =  1


KeyboardInterrupt: ignored